## Import

In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

import numpy as np
import pandas as pd 
import selenium
import warnings
import time
import re

warnings.filterwarnings(action='ignore')

## Get information

In [2]:
URL = 'https://securities.koreainvestment.com/main/mall/openels/EdlsInfo.jsp?cmd=TF02cc000000_Main'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=0.5)

In [3]:
driver.get(url=URL)

In [4]:
els_checkbox = driver.find_elements_by_css_selector("label.mtl_checkbox.marL20")
els_checkbox[0].click()

In [5]:
start_time, end_time = driver.find_elements_by_css_selector("td.t_left > div.calendarWap > div.mtl_calendar > a.btn_calendar")

In [6]:
start_time.click()
time.sleep(1)

* set start years

In [7]:
year_list = driver.find_elements_by_css_selector("a.year")
year_list[0].click()
time.sleep(1)

year_list_2 = driver.find_elements_by_css_selector("div.select_year.on > a")

target_year = 2010
target_idx =[idx for idx, x in enumerate(year_list_2) if str(target_year) == x.text]

time.sleep(0.5)
year_list_2[target_idx[0]].click()


okey = driver.find_elements_by_css_selector("button.mtl_button.btnOk")
okey[0].click()
time.sleep(1)

* set end years

In [8]:
end_time.click()
time.sleep(1)

In [9]:
year_list = driver.find_elements_by_css_selector("div.selected_date > a.year")
year_list[1].click()
time.sleep(1)

year_list_2 = driver.find_elements_by_css_selector("div.select_year.on > a")
target_year = 2021

target_idx =[idx for idx, x in enumerate(year_list_2) if str(target_year) == x.text]
time.sleep(0.5)
year_list_2[target_idx[0]].click()


okey = driver.find_elements_by_css_selector("button.mtl_button.btnOk")
okey[1].click()
time.sleep(1)

* 상환 확정 여부

In [10]:
'''
temp_btn = driver.find_elements_by_css_selector("td.t_left > span.mtl_selectbox")
temp_btn[1].click()
time.sleep(1)

driver.find_elements_by_css_selector("ul.selectList > li > a")[1].click()
time.sleep(1)
'''

'\ntemp_btn = driver.find_elements_by_css_selector("td.t_left > span.mtl_selectbox")\ntemp_btn[1].click()\ntime.sleep(1)\n\ndriver.find_elements_by_css_selector("ul.selectList > li > a")[1].click()\ntime.sleep(1)\n'

* click search button

In [11]:
search_button = driver.find_elements_by_css_selector("div.btnCenter > button.mtl_button")
search_button[0].click()

* set to showing 120 products

In [12]:
product_list = driver.find_elements_by_css_selector("span.mtl_selectbox.marL10")
product_list[0].click()

In [13]:
time.sleep(1)
product_list2 = driver.find_elements_by_css_selector("ul.selectList > li > a")
product_list2[3].click()

* product 정보 수집

In [14]:
# x_path 방법 탐색 필요
shorts_list = driver.find_elements_by_css_selector("td.t_left > span.short_yellow")
title_list = driver.find_elements_by_css_selector("td.t_left > a.product_listTitle.name")

* 차선책

In [15]:
title_list = []
product_list = []
law_list = []
profit_list = []
due_list = []
bools_list = []
date_list = []
bools2_list = []

* crawling code

In [16]:
total_products = int(driver.find_element_by_css_selector("em.fColor.fBd.letter_0").text)
pages_loop = np.int(np.ceil(total_products / 120))
print(total_products)

8797


In [17]:
for cnt in tqdm(range(1, pages_loop+1)):
    time.sleep(1)
    
    ## PRODUCT TABLE
    row_list = driver.find_elements_by_css_selector("div.tableDefault2 > table > tbody > tr")

    for row in row_list:
        _, _, title, product, law, profit, _, due, bools, date, bools2 = [x.text for x in row.find_elements_by_css_selector("td")]
        title_list.append(title)
        product_list.append(product)
        law_list.append(law)
        try:
            profit_list.append(np.float(re.sub("[^0-9.]","",profit)))
        except ValueError:
            profit_list.append(-10)
        due_list.append(due)
        bools_list.append(bools)
        date_list.append(pd.to_datetime(date))
        bools2_list.append(bools2)
        
    if cnt == pages_loop:
        continue
        
    ## PAGE LIST
    number_list = driver.find_elements_by_css_selector("span.num > a")

    ## go to next page
    if cnt % 10 != 0:
        number_list[cnt % 10].click()
    ## go to next page list
    else:
        driver.find_elements_by_css_selector("a.btn.next")[0].click()

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [44:01<00:00, 35.70s/it]


* save csv file

In [18]:
title_list = [x.split("\n") for x in title_list]
product_list = [x.split("\n") for x in product_list]

In [19]:
product_df = pd.DataFrame(title_list, columns = ["short", "title"])
temp_df = pd.DataFrame({"products":product_list,"law": law_list, "profit": profit_list, "due": due_list, "bools": bools_list, "date":date_list,"bools2_list":bools2_list})

In [20]:
els_df = pd.concat([product_df, temp_df], axis = 1)

In [21]:
els_df.head()

,short,title,products,law,profit,due,bools,date,bools2_list
0,원금비보장종목형,TRUE ELS 제13653 회 (월지급식 스텝다운)[온라인전용상품],"[AMD, S&P500]",85-85-85-80-80-70/50(종가),8.16,3년/6개월,비보장,2021-02-03,미상환
1,원금비보장종목형,TRUE ELS 제13652 회 (멀티배리어리자드)[온라인전용상품],"[삼성전자, NAVER, S&P500]","92-90-87-82-80-77/52(종가),리자드배리어87%,82%",8.70,3년/6개월,비보장,2021-02-03,상환
2,원금비보장종목형,TRUE ELS 제13651 회 (스텝다운),"[APPLE, S&P500]",85-85-85-80-80-70/45(종가),6.00,3년/6개월,비보장,2021-02-03,상환
3,원금비보장종목형,TRUE ELS 제13650 회 (멀티배리어리자드),"[삼성전자, SK하이닉스, S&P500]","92-90-87-85-82-77/52(종가),리자드배리어87%,82%",6.10,3년/6개월,비보장,2021-02-03,상환
4,원금비보장종목형,TRUE ELS 제13649 회 (스텝다운),"[삼성전자, S&P500, EUROSTOXX50]",88-88-82-82-80-77/48(종가),5.00,3년/6개월,비보장,2021-02-03,상환


In [68]:
els_df.products = ["--".join(x) for x in els_df.products]

In [70]:
els_df.to_csv("els_df.csv", index = False, encoding = "cp949")

In [23]:
np.save("product_list.npy",product_list)

In [114]:
re.sub("","",els_df.title[0])

'TRUE ELS 제13653 회 [온라인전용상품]'

In [10]:
els_df = pd.read_csv("els_df.csv", encoding = "cp949")
product_list = [x.split("--") for x in els_df.products]

* 종목 추출

In [11]:
unique_pro = []

for product in product_list:
    unique_pro.extend(product)

In [12]:
pd.Series(unique_pro).unique()

array(['AMD', 'S&P500', '삼성전자', 'NAVER', 'APPLE', 'SK하이닉스', 'EUROSTOXX50',
       'HSCEI', 'KOSPI200', 'NIKKEI225', '현대차', 'FACEBOOK', 'NVIDIA',
       'AMAZON', 'NETFLIX', 'EUROSTOXX BANKS', 'POSCO', '셀트리온', 'HSI',
       '삼성전자 보통주', '삼성전자보통주', 'SPXLTBUP', 'EUROSTOXX5O', 'KOSDAQ150',
       'HSI(항셍지수)', 'MICROSOFT', 'DAX', 'ERUOSTOXX50', '기아차', '신한지주',
       '현대차 보통주', 'S&P500I', 'FTSE100', 'EUROSTOXX', 'EUOSTOXX50',
       'EUROSTOXX500', '3지수 월지급식 스텝다운형', '대한항공', 'LG생활건강', '삼성중공업',
       'SK텔레콤', '현대모비스', 'LG유플러스', '현대미포조선', '삼성SDI', '현대제철', '롯데케미칼',
       'KB금융', 'LG디스플레이', '한국전력', '현대중공업', '삼성엔지니어링', 'KOSP200', '삼성전기',
       '고려아연', 'LG디스플레이 보통주', '현대제철 보통주', 'LG전자', '현대글로비스', '기아차 보통주',
       'GS건설 보통주', 'KT', '하나금융지주', '엔씨소프트', 'SK', 'KOSPI200 지수', '한국가스공사',
       '대우증권', '삼성카드', 'GS건설', '두산중공업', '대우조선해양', 'NHN', 'SK이노베이션',
       '대림산업', 'LG', 'KOSPI200지수', '우리금융', 'HSCEI지수', 'LG화학', '호남석유',
       'GS', '삼성물산'], dtype=object)

In [13]:
index_list = ['HSCEI', 'KOSPI200', 'NIKKEI225', 'EUROSTOXX50', 'HSI', 
              'SPXLTBUP', 'EUROSTOXX BANKS', 'EUROSTOXX5O', 'KOSDAQ150', 
              'HSI(항셍지수)', 'DAX', 'ERUOSTOXX50', 'S&P500', 'KOSPI200지수', 'S&P500I',
             'FTSE100', 'EUROSTOXX', 'EUOSTOXX50', 'EUROSTOXX500', '3지수 월지급식 스텝다운형',
             'KOSP200', 'KOSPI200 지수', 'HSCEI지수']
stock_list = [x for x in pd.Series(unique_pro).unique() if x not in index_list]
print(stock_list)

['AMD', '삼성전자', 'NAVER', 'APPLE', 'SK하이닉스', '현대차', 'FACEBOOK', 'NVIDIA', 'AMAZON', 'NETFLIX', 'POSCO', '셀트리온', '삼성전자 보통주', '삼성전자보통주', 'MICROSOFT', '기아차', '신한지주', '현대차 보통주', '대한항공', 'LG생활건강', '삼성중공업', 'SK텔레콤', '현대모비스', 'LG유플러스', '현대미포조선', '삼성SDI', '현대제철', '롯데케미칼', 'KB금융', 'LG디스플레이', '한국전력', '현대중공업', '삼성엔지니어링', '삼성전기', '고려아연', 'LG디스플레이 보통주', '현대제철 보통주', 'LG전자', '현대글로비스', '기아차 보통주', 'GS건설 보통주', 'KT', '하나금융지주', '엔씨소프트', 'SK', '한국가스공사', '대우증권', '삼성카드', 'GS건설', '두산중공업', '대우조선해양', 'NHN', 'SK이노베이션', '대림산업', 'LG', '우리금융', 'LG화학', '호남석유', 'GS', '삼성물산']


In [120]:
# np.save("stock_list.npy",stock_list)

* 종목이 들어간 ELS 찾기, 시작 일자

In [14]:
def check_tf(stock, df):
    return [True if stock in x else False for x in df.products]

In [15]:
def total_check_tf(stock_list, df):
    base_idx = np.array([False for x in range(df.shape[0])])
    for stock in stock_list:
        base_idx += np.array(check_tf(stock, df))
    return list(base_idx)

In [16]:
stock_idx = total_check_tf(stock_list, els_df)

In [17]:
els_df.shape

(8880, 9)

* 종목 포함시

In [36]:
total_cnt = els_df.loc[stock_idx,:].shape[0]; total_cnt

1485

In [33]:
# 종목이 들어간 profit
profit_1 = els_df.loc[stock_idx, 'profit'].mean(); profit_1

10.721919191919245

In [38]:
from collections import Counter

temp_dict = Counter(els_df.loc[stock_idx, 'bools2_list']); temp_dict

Counter({'미상환': 408, '상환': 1077})

In [44]:
# 미상환율
rate1 = temp_dict['미상환']/total_cnt
rate1

0.27474747474747474

* 종목 미포함시

In [40]:
total_cnt2 = els_df.loc[[~x for x in stock_idx],:].shape[0]

In [41]:
# 종목이 들어가지 않은 profit
profit_2 = els_df.loc[[~x for x in stock_idx], 'profit'].mean(); profit_2

6.6534915483434744

In [42]:
temp_dict2 = Counter(els_df.loc[[~x for x in stock_idx], 'bools2_list'])
temp_dict2

Counter({'미상환': 1169, '상환': 6226})

In [45]:
# 미상환율
rate2 = temp_dict2['미상환']/total_cnt2
rate2

0.15807978363759298

* 비교

In [46]:
# 종목이 포함된 상품의 상대적 미상환율
rate1 / rate2

1.7380304326412108

In [48]:
# 종목이 포함된 상품의 상대적 수익률 
profit_1 / profit_2

1.6114725800754477